In [1]:
# import packages
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace

# set up AzureML workspace
config_path = '../utils/config.json'
tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47'  # this is outputted post `az login`
interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)  # create log-in object
ws = Workspace.from_config(path=config_path, auth=interactive_auth)  # link workspace


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [2]:
# import packages
import shutil
# from sklearn.ensemble import IsolationForest

# # copy over script for model creation
# shutil.copy('../src/triton_isolation_forest.py', '.')
# from triton_isolation_forest import *

# # preprocess data
# url = 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz'  # URL of the public dataset
# df = download_data(url)  # read data from the url
# df = process_data(df)  # subset the data and select top labels
# # split data
# train, test_norm, test_anom = split_data(df)
# X_train, y_train = train  # unpack training data
# # train model
# model = IsolationForest(random_state=RAND_STATE)
# model.fit(X_train)
# # convert into ONNX format
# from skl2onnx import convert_sklearn
# from skl2onnx.common.data_types import FloatTensorType, Int64TensorType
# initial_types = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
# onx = convert_sklearn(model, initial_types=initial_types)
# with open('../src/isolation_forest.onnx', 'wb') as f:
#     f.write(onx.SerializeToString())


In [3]:
# import packages
import os

# define helper functions
def instantiate_models(src_file):
    '''
    creates the expected triton directory system given a source model file
    '''
    # extract model name and source file name
    src_name = os.path.basename(src_file)
    model_name = src_name.split('.')[0]  # assumes a <model>.onnx structure
    # set outputted directory name and model filename
    dir_name = f'models/triton/{model_name}/1/'
    file_name = f'{dir_name}{src_name}'
    # copies the data over
    os.makedirs(dir_name, exist_ok=True)  # instantiate directory
    shutil.copy(src_file, file_name)  # copy model onnx file
    # return the file/directory locations
    return dir_name, file_name

def remove_models():
    '''
    removes the models directory where the triton inputs are stored
    '''
    shutil.rmtree('models')


In [4]:
# import packages
from azureml.core.model import Model

# setup model for registration
dir_name,file_name = instantiate_models('../src/isolation_forest.onnx')
# register model
model = Model.register(
    model_path='models',
    model_name='IsolationForestTutorial',
    description='Anomaly detection for KDD network intrusion dataset',
    workspace=ws,
    model_framework=Model.Framework.MULTI,
)


Registering model IsolationForestTutorial


In [5]:
# import packages
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Environment

# setup environment (obtain environment.yml from `conda env export > environment.yml`)
env_name,env_path = 'SampleEnv', '../utils/environment.yml'
env = Environment.from_conda_specification(name=env_name, file_path=env_path)

# setup service for deployment
service_name = 'triton-isolation-forest'
infer_config = InferenceConfig('../utils/entry_script.py', environment=env)
deploy_config = LocalWebservice.deploy_configuration(port=6789)
# deploy service
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=infer_config,
    deployment_config=deploy_config,
    overwrite=True,
)
# check deployment status
service.wait_for_deployment(show_output=True)


Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.
Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.


Generating Docker build context.
2021/05/19 20:13:29 Downloading source code...
2021/05/19 20:13:30 Finished downloading source code
2021/05/19 20:13:31 Creating Docker network: acb_default_network, driver: 'bridge'
2021/05/19 20:13:32 Successfully set up Docker network: acb_default_network
2021/05/19 20:13:32 Setting up Docker configuration...
2021/05/19 20:13:33 Successfully set up Docker configuration
2021/05/19 20:13:33 Logging in to registry: 27ebdbdbb34a4186a9c6cf92029c3be2.azurecr.io
2021/05/19 20:13:34 Successfully logged into 27ebdbdbb34a4186a9c6cf92029c3be2.azurecr.io
2021/05/19 20:13:34 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/05/19 20:13:34 Scanning for dependencies...
2021/05/19 20:13:35 Successfully scanned dependencies
2021/05/19 20:13:35 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/18 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:2021012

Removing intermediate container 71d13bcb293f
 ---> 0551fd142131
Step 9/18 : ENV PATH /azureml-envs/azureml_85d71eb43e0985ef03c127d2ca5e6476/bin:$PATH
 ---> Running in 2af266fa561c
Removing intermediate container 2af266fa561c
 ---> dde72b5b2bf8
Step 10/18 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> e78c6cc35a4b
Step 11/18 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> 0ac3c50fbf3d
Step 12/18 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_85d71eb43e0985ef03c127d2ca5e6476
 ---> Running in b256facbd0f2
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with materialized conda environment details
Successfully sent materialized environment details
Removing intermediate container b256facbd0f2
 ---> 76bbf850441b
Step 13/18 : ENV AZUREML

Step 1/5 : FROM 27ebdbdbb34a4186a9c6cf92029c3be2.azurecr.io/azureml/azureml_e09bfaa27865cae0196655c08e040dc7
 ---> baad9d71e345
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> bed8e51bb602
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjY1NjA1NzVkLWZhMDYtNGU3ZC05NWZiLWY5NjJlNzRlZmQ3YSIsInJlc291cmNlR3JvdXBOYW1lIjoidXdfYW5vbWFseWRldGVjdGlvbiIsImFjY291bnROYW1lIjoiYW5vbWFseWRldGVjdGlvbiIsIndvcmtzcGFjZUlkIjoiMjdlYmRiZGItYjM0YS00MTg2LWE5YzYtY2Y5MjAyOWMzYmUyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in c88270c2aa49
 ---> a5241fc6e073
Step 4/5 : RUN mv '/var/azureml-app/tmplyokb84y.py' /var/azureml-app/main.py
 ---> Running in 83585b5cd573
 ---> 95ed7e5c3b9e
Step 5/5 : CMD ["runsvdir","/var/runit"]
 ---> Running in ee499c7a8763
 ---> 173b98f5c5c1
Successfully built 173b98f5c5c1
Successfully tagged triton-isolation-forest:latest
Container (name:boring_lalande, id:3e

Error: Container has crashed. Did your init method fail?




Container Logs:
2021-05-19T20:22:18,336822241+00:00 - gunicorn/run 
2021-05-19T20:22:18,336822382+00:00 - iot-server/run 
2021-05-19T20:22:18,341378897+00:00 - rsyslog/run 
2021-05-19T20:22:18,343051068+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-05-19T20:22:18,679336435+00:00 - iot-server/finish 1 0
2021-05-19T20:22:18,681129796+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 40
SPARK_HOME not set. Skipping PySpark Initialization.
Exception in worker process
Traceback (most recent call last):
  File "/azureml-envs/azureml_85d71eb43e0985ef03c127d2ca5e6476/lib/python3.8/site-packages/gunicorn/arbiter.py", line 589, in spawn_worker
    worker.init_process()
  File "/azureml-envs/azureml_85d71eb43e0985ef03c127d2ca5e6476/lib/python3.8/site-packages/gunicorn/workers/base.py", line 134, in

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
print(service.get_logs())
